<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/NLP/4_Seq2Seq/ejercicios/ejercicios_solucion.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Ejercicio Clase 4

En este notebook modificaremos el encoder del modelo de traducción automática visto en la clase para utilizar una RNN bidireccional. Al igual que en el modelo anterior, usamos un GRU de dos capas, sin embargo ahora la haremos bidireccional. Con un RNN bidireccional, tenemos dos RNN en cada capa. Un *RNN hacia adelante* que repasa los embedding de la oración de izquierda a derecha (que se muestra a continuación en verde), y un *RNN hacia atrás* que repasa los embedding de la oración de derecha a izquierda (verde azulado). Todo lo que necesitamos hacer en el código es establecer `bidirectional = True` y luego pasar los embedding de la oración al RNN como antes.

![](https://i.imgur.com/bdUnvj9.png)

Ahora tenemos:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Donde $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

Como antes, solo pasamos una entrada(`embedded`) al RNN, que le dice a PyTorch que inicialice los estados ocultos iniciales hacia adelante y hacia atrás ($ h_0 ^ \rightarrow $ y $ h_0 ^ \leftarrow $, respectivamente) como un tensor de todos los ceros. También obtendremos dos vectores de contexto, uno del RNN hacia adelante después de haber visto la última palabra en la oración, $ z ^ \rightarrow = h_T ^ \rightarrow $, y uno del RNN hacia atrás después de haber visto la primera palabra en la oración, $ z ^ \leftarrow = h_T ^ \leftarrow $.

El RNN devuelve `outputs` y `hidden`.

`outputs` es de tamaño `[longitud de la oración de origen, tamaño de lote, (dimensión de variables ocultas)x(numero de direcciones)]` donde los primeros ` num_hiddens` elementos en el tercer eje son los estados ocultos de la capa superior hacia adelante RNN, y los últimos `num_hiddens ` elementos son estados ocultos de la capa superior hacia atrás RNN. Podemos pensar en el tercer eje como los estados ocultos hacia adelante y hacia atrás concatenados entre sí, es decir, $ h_1 = [h_1 ^ \rightarrow; h_{T} ^ \leftarrow] $, $ h_2 = [h_2 ^ \rightarrow; h_{T-1} ^ \leftarrow] $ y podemos denotar todos los estados ocultos del encoder (hacia adelante y hacia atrás concatenados juntos) como $ H = \{h_1, h_2, ..., h_T \} $. 

`hidden` es de tamaño **[número de capas * número de direcciones, tamaño de lote, dimensión de variables ocultas]**, donde `hidden[- 2,:,:]` entregaría el estado oculto de la capa superior de la RNN hacia adelante después del paso de tiempo final (es decir, después de haber visto la última palabra en la oración), `hidden[- 1,:,: ]` entregaría el estado oculto de la capa superior de la RNN hacia atrás después del paso de tiempo final (es decir, después de haber visto la primera palabra en la oración) y así sucesivamente.

Como el decoder no es bidireccional, solo necesitaremos un vector de contexto $z$, y actualmente tenemos dos versiones, uno hacia adelante y uno hacia atrás ($ z ^ \rightarrow = h_T ^ \rightarrow $ y $ z ^ \leftarrow = h_T ^ \leftarrow $, respectivamente). Resolvemos esto concatenando los dos vectores de contexto juntos, pasándolos a través de una capa lineal, $ g $, y aplicando la función de activación $ \tanh $. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

Lo mismo sucede con los estados ocultos que pasamos para usar como estado inicial $s_0$ en el decoder. Aquí también definiremos una capa lineal para transformarlos, pero teniendo en cuenta que tendremos que hacerlo capa por capa.


## Ejercicio 1

Teniendo en cuenta las cuestiones de implementación mencionadas en la introducción, modificar el Encoder y el Decoder planteados en la clase 4 para que se procese a la oración de origen con una RNN bidireccional.

In [ ]:
# inserte su código aquí

In [ ]:
import torch
import torch.nn as nn
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()

    # Más tarde puede haber argumentos adicionales
    # (por ejemplo, longitud para excluir el relleno)
    def forward(self, X, *args):
        raise NotImplementedError

In [ ]:
class Decoder(nn.Module):
    """The base decoder interface for the encoder-decoder architecture."""
    def __init__(self):
        super().__init__()

    # Más tarde puede haber argumentos adicionales
    # (por ejemplo, longitud para excluir el relleno)
    def init_state(self, enc_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

In [ ]:
def init_RNN(module):
    if type(module) == nn.Linear:
         nn.init.xavier_uniform_(module.weight)
    if type(module) == nn.GRU:
        for param in module._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(module._parameters[param])

class RNNEncoder(Encoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, dec_num_hiddens,
                 num_layers, dropout=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers,
                          bidirectional = True, dropout=dropout)
        self.fc = nn.Linear(num_hiddens * 2, dec_num_hiddens)
        self.hid_dim = num_hiddens
        self.n_layers = num_layers
        self.apply(init_RNN)

    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.embedding(src)
        
        #embedded = [src len, batch size, embed_size]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, num_hiddens * num directions]
        #hidden = [num_layers * num directions, batch size, num_hiddens]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        outputs = torch.tanh(self.fc(outputs[-1]))
        #outputs = [batch size, dec_num_hiddens]
        
        hid_layers = []
        for l in range(self.n_layers):
            hid_forward = hidden[2*l,:,:]
            #hid_forward = [batch size, num_hiddens]
            hid_backward = hidden[2*l+1,:,:]
            #hid_backward = [batch size, num_hiddens]
            hid_layer = torch.tanh(self.fc(torch.cat((hid_forward, hid_backward), dim = 1)))
            #hid_layer = [batch size, dec_num_hiddens]
            hid_layers.append(hid_layer.unsqueeze(0))
        
        hidden = torch.cat(hid_layers, dim = 0)
        #hidden = [num_layers, batch size, dec_num_hiddens]
        
        return outputs, hidden

In [ ]:
class RNNDecoder(Decoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size+num_hiddens, num_hiddens,
                           num_layers, dropout=dropout)
        self.linear = nn.Linear(num_hiddens, vocab_size)
        self.hid_dim = num_hiddens
        self.n_layers = num_layers
        self.output_dim = vocab_size
        self.apply(init_RNN)

    def init_state(self, enc_output, *args):
        #se queda con el output del último paso de tiempo
        return enc_output

    def forward(self, X, context, hidden):
        # X shape: (batch_size)
        # context shape: (batch_size, num_hiddens)
        # hidden shape: (n layers, batch_size, num_hiddens)

        input = X.unsqueeze(0)
        context = context.unsqueeze(0)
        #input shape: (1,batch_size)
        #context shape: (1, batch_size, num_hiddens)

        embs = self.embedding(input)
        # embs shape: (1, batch_size, embed_size)

        # Concatena el token de entrada con el contexto
        embs_and_context = torch.cat((embs, context), -1)
        #embs_and_context shape: (1, batch size, embed_size + num_hiddens)
        
        #Genera las salidas de la RNN
        rnn_outputs, state = self.rnn(embs_and_context, hidden)
        # recuerde que output devolverá siempre el valor de las variables ocultas (de todas las direcciones) de la última capa
        # por cada token en la secuencia de cada elemento del lote (seq_len, batch size, hid dim * n directions) .
        # rnn_outputs shape: (1, batch size, hid dim) 
        # recuerde que state devolverá siempre el valor de las variables ocultas de todas las capas 
        # (y en cada dirección si hay mas de una) de cada elemento del lote. (n layers * n directions, batch_size, num_hiddens) 
        # state shape: (num_layers, batch_size, num_hiddens)       

        #Genera las probabilidades de cada token en el vocabulario
        outputs = self.linear(rnn_outputs.squeeze(0))
        #outputs = torch.cat((embs.squeeze(0), rnn_outputs.squeeze(0), context),dim = 1)

        # outputs shape: (batch_size, vocab_size)
        return outputs, state

## Ejercicio 2 

Cree un modelo Seq2Seq que utilice el encoder y el decoder del ejercicio anterior. Entrénelo sobre el dataset inglés-español utilizado en clase

In [ ]:
###################################### MODELO SEQ2SEQ #######################################
import random
class Seq2Seq_TF(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "¡Las dimensiones ocultas del encoder y del decoder deben ser iguales!"
        assert encoder.n_layers == decoder.n_layers, \
            "¡El número de capas del encoder y del decoder deben ser iguales!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        enc_outputs, enc_hidden = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]

        context = self.decoder.init_state(enc_outputs)
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and context
            dec_output, dec_hidden = self.decoder(input, context, enc_hidden)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = dec_output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = dec_output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
############################### CARGA DE DATOS #################################
import os.path
import re
from shutil import unpack_archive
import random
from collections import Counter
import torch
from torchtext.vocab import vocab
from torch.utils.data.sampler import Sampler
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

data = None
!wget -O spa-eng.zip http://www.manythings.org/anki/spa-eng.zip
if not os.path.isfile("spa.txt"):
    unpack_archive('./spa-eng.zip', extract_dir='./', format='zip')
with open('./spa.txt', encoding='utf-8') as f:
    data = f.read()
    data = re.sub("\tCC-BY 2\.0.*","",data) # acá elimino información adicional
    data = re.sub(r"[\u202f]|[\xa0]"," ",data) # aca saco caracteres raros
    data = re.sub("([,\.:;!?])"," \\1",data) # aca  y abajo tokenizo puntuación
    data = re.sub("([¡¿])","\\1 ",data).lower()


SRC_IDX, TGT_IDX = 0, 1
SEED = 12312

data2 = data.split('\n')
random.seed(SEED)
random.shuffle(data2)

data_list = []
for i, line in enumerate(data2):
    parts = line.split('\t')
    if len(parts) == 2:
        # Skip empty tokens
        new_src = [t for t in f'{parts[SRC_IDX]} <eos>'.split(' ') if t]
        new_tgt = [t for t in f'<bos> {parts[TGT_IDX]} <eos>'.split(' ') if t]
        length_src = len(new_src)
        data_list.append((new_src, length_src, new_tgt))
        
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n = len(data_list)
split1, split2 = int(0.7*n), int(0.9*n)
train_list = data_list[:split1]
val_list = data_list[split1:split2]
test_list = data_list[split2:]

counter_src, counter_tgt = Counter(), Counter()
for i in range(len(train_list)):
  counter_src.update(train_list[i][0])
  counter_tgt.update(train_list[i][-1])

vocab_src = vocab(counter_src, min_freq = 2,
              specials=('<unk>', '<eos>', '<bos>', '<pad>'))
vocab_src.set_default_index(vocab_src['<unk>'])

vocab_tgt = vocab(counter_tgt, min_freq = 2,
              specials=('<unk>', '<eos>', '<bos>', '<pad>'))
vocab_tgt.set_default_index(vocab_tgt['<unk>'])

class BucketSampler(Sampler):

    def __init__(self, batch_size, train_list):
        self.length = len(train_list)
        self.train_list = train_list
        self.batch_size = batch_size
        indices = [(i, s[1]) for i, s in enumerate(self.train_list)]
        random.seed(SEED)
        random.shuffle(indices)
        pooled_indices = []
        # creamos minilotes de tamaños similares
        for i in range(0, len(indices), batch_size * 100):
            pooled_indices.extend(sorted(indices[i:i + batch_size * 100],
                                         key=lambda x: x[1], reverse=True))

        self.pooled_indices = pooled_indices

    def __iter__(self):
        for i in range(0, len(self.pooled_indices), self.batch_size):
            yield [idx for idx, _ in self.pooled_indices[i:i + self.batch_size]]

    def __len__(self):
        return (self.length + self.batch_size - 1) // self.batch_size

SRC_PAD_IDX = vocab_src['<pad>']
TGT_PAD_IDX = vocab_tgt['<pad>']

def collate_batch(batch):
    text_src, length_list, text_tgt_in, text_tgt_out = [], [], [], []
    for (src, length, tgt) in batch:
        # convertimos el texto en tokens
        processed_src = torch.tensor([vocab_src[token] for token in src])
        processed_tgt = torch.tensor([vocab_tgt[token] for token in tgt])
        text_src.append(processed_src)
        text_tgt_in.append(processed_tgt[:-1])
        text_tgt_out.append(processed_tgt[1:])
        # guardamos la longitud de cada token
        length_list.append(length)
    # armamos la tupla que conformara un ejemplo de minilote.
    result = (pad_sequence(text_src, padding_value=SRC_PAD_IDX),
              pad_sequence(text_tgt_in, padding_value=TGT_PAD_IDX),
              pad_sequence(text_tgt_out, padding_value=TGT_PAD_IDX),)
    return result

batch_size = 64  # A batch size of 64

train_bucket = BucketSampler(batch_size, train_list)
train_iter = DataLoader(train_list,
                          batch_sampler=train_bucket,
                          collate_fn=collate_batch)

val_bucket = BucketSampler(batch_size, val_list)
val_iter = DataLoader(val_list,
                          batch_sampler=val_bucket,
                          collate_fn=collate_batch)

test_bucket = BucketSampler(batch_size, test_list)
test_iter = DataLoader(test_list,
                          batch_sampler=test_bucket,
                          collate_fn=collate_batch)

In [ ]:
############################### BUCLE DE ENTRENAMIENTO #################################
import time
import math
import torch.optim as optim

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src, tgt_input, tgt_out = batch
        src, tgt_input, tgt_out = src.to(device), tgt_input.to(device), tgt_out.to(device)        
        #src: son las frases en el idioma origen que le pasaremos como entrada al encoder
        #src.shape : [src len, batch size]
        #tgt_input: son las frases en el idioma destino que le pasaremos como entrada al decoder (con `<bos>` como primer token y sin `<eos>`)
        #tgt_out: son las frases en el idioma destino que usaremos para calcular la pérdida (con `<eos>` como finalizador de oración y sin `<bos>`)
        #tgt.shape : [trg len, batch size]
        
        optimizer.zero_grad()
        output = model(src, tgt_input)
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        #como la función de pérdida solo funciona en entradas 2d con objetivos 1d,
        # necesitamos aplanar cada una de ellas con .view
        output = output[1:].view(-1, output_dim)
        trg = tgt_out[1:].view(-1)
        #trg = [trg len * batch size]
        #output = [trg len * batch size, output dim]
        
        #calculamos los gradientes
        loss = criterion(output, trg)
        loss.backward()

        #recortamos los gradientes para evitar que exploten
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0 
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, tgt_input, tgt_out = batch
            src, tgt_input, tgt_out = src.to(device), tgt_input.to(device), tgt_out.to(device)    
            output = model(src, tgt_input, 0) #turn off teacher forcing
            #output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]

            output = output.view(-1, output_dim)
            trg = tgt_out.view(-1)
            #trg = [trg len * batch size]
            #output = [trg len * batch size, output dim]

            loss = criterion(output, trg)            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



INPUT_DIM = len(vocab_src.get_itos())
OUTPUT_DIM = len(vocab_tgt.get_itos())
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = RNNEncoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = RNNDecoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq_TF(enc, dec, device).to(device)

model = Seq2Seq_TF(enc, dec, device).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = TGT_PAD_IDX)

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')